In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os


# Define the column names, start points, end points, column types, and number of decimal places
columns_lo_alt = [
    ('blank', 1, 1, 'char', 0),
    ('flight_num', 2, 8, 'char', 0),
    ('blank', 9, 15, 'char', 0),
    ('date_YY.YYY', 16, 19, 'long', 3),
    ('colatitude', 20, 23, 'long', 3),
    ('longitude', 24, 27, 'long', 3),
    ('mag_declination_calc_deg', 28, 31, 'long', 3),
    ('mag_inclination_calc_deg', 32, 35, 'long', 3),
    ('mag_horizontal_intensity_calc_nT', 36, 39, 'long', 0),
    ('mag_north_component_obs_nT', 40, 43, 'long', 0),
    ('mag_east_component_obs_nT', 44, 47, 'long', 0),
    ('mag_vertical_intensity_obs_nT', 48, 51, 'long', 0),
    ('mag_total_intensity_obs_nT', 52, 55, 'long', 0),
    ('alt_radar_m', 56, 59, 'long', 0),
    ('data_collection_code', 60, 61, 'short', 0),
    ('source_id', 62, 65, 'long', 0),
    ('serial_num', 66, 69, 'long', 0),
    ('mag_element_code', 70, 76, 'char', 0),
    ('time_gmt_HHMM', 77, 80, 'long', 0),
    ('country_code', 81, 84, 'long', 0)
]


columns_hi_lat = [
    ('time_gmt_sec', 1, 4, 'long', 4),
    ('latitude', 5, 8, 'long', 4),
    ('longitude', 9, 12, 'long', 4),
    ('mag_east_component_obs_nT', 13, 16, 'long', 0),
    ('mag_north_component_obs_nT', 17, 20, 'long', 0),
    ('mag_horizontal_intensity_calc_nT', 21, 24, 'long', 0),
    ('mag_vertical_intensity_obs_nT', 25, 28, 'long', 0),
    ('mag_total_intensity_calc_nT', 29, 32, 'long', 0),
    ('mag_declination_calc_deg', 33, 36, 'long', 4),
    ('mag_inclination_calc_deg', 37, 40, 'long', 4),
    ('mag_total_intensity_obs_nT', 41, 44, 'long', 0),
    ('alt_barometric_ft', 45, 48, 'long', 0)
]



# Read the file into a DataFrame
def read_binary_file(filename, columns):
    data = []
    with open(filename, 'rb') as f:
        # Skip the header
        f.seek(128)
        
        while True:
            record = []
            for name, start, end, data_type, decimal_places in columns:
                if data_type == "char":
                    value = f.read(end - start + 1).decode("utf-8").strip()
                elif data_type == "short":
                    value = int.from_bytes(f.read(2), byteorder='little')
                elif data_type == "long":
                    value = int.from_bytes(f.read(4), byteorder='little')
                    if decimal_places > 0:
                        value /= 10 ** decimal_places
                record.append(value)
                        
            if record[2]=='':
                break
            else:
                data.append(record)

    return pd.DataFrame(data, columns=[column[0] for column in columns])




In [ ]:
directory = '/Users/simon/Data/ProjectMagnet/geomag/cdroms/Project_Magnet/lo_alt/pm_53_70'

bin_files = [filename for filename in os.listdir(directory) if filename.endswith('.bin')]

#print(bin_files)

df_list = []

fig,ax = plt.subplots()
for filename in bin_files: 
    file_path = '{:s}/{:s}'.format(directory, filename)
    df = read_binary_file(file_path, columns_lo_alt)
    ax.plot(df.longitude, 90.-df.colatitude, '.')
    
    df_list.append(df)

plt.show()


In [ ]:
df_lo_alt = pd.concat(df_list)

df_lo_alt.columns

In [ ]:
df_lo_alt.mag_total_intensity_obs_nT.hist(bins=50)
